In [1]:
import pyodbc
import jieba

from math import*
import codecs
from nltk import word_tokenize
import os #讀取程式目前路徑用
import sys #為了能夠吃進主程式外部輸進的參數
import nltk #引入自然語言處理模組
import io #檔案io
from nltk.corpus import stopwords #去無用字
import re #需要用正則表示式移除標點與數字
from nltk.corpus import wordnet as wn #2013/10/26改用wordnet來做字根還原
from nltk.tokenize import word_tokenize
import json
import numpy as np
import numpy.linalg as LA
import string
import csv
from __future__ import print_function, unicode_literals
import sys
sys.path.append("../")
import jieba
import jieba.posseg as pseg
print('-----start-----')
#連接自定義字典
print('import the customer dictionary....')
jieba.load_userdict("dict_policy.txt")
print('*sccessful import*')

#連接資料庫
print("try to link with DataBase:cate_test......")
cnxn = pyodbc.connect("DRIVER={SQL Server};SERVER=LEAVES-PC\SA000;DATABASE=cate_test;UID=sa;PWD=00000000")
cursor = cnxn.cursor()
print("*sccessful link*")

#選擇資料集
cursor.execute("select [BulletinPlatform] from [cate_test].[dbo].[six_city_20161020001]" )

#建立資料集
print('setting up the data.......')
results = cursor.fetchall()
print('*sccessful set up*')
#print(results)

#####製作所有詞變成list
f= open('diction_split.txt', encoding = 'utf8')#讀所有詞
s = f.read()
#print(s)
spilt_s = s.split("\n")#分所有詞
#print('所有詞=',spilt_s)
#print(type(spilt_s))

new_gene=[]

def delpunc_n_jieba_n_tok(re): 
    count=0
    for i in re:
        count+=1
        #print ('%d: %s' % (count, i))
    for times in range(0,count):
        chineseFilter1 =  [u'．',u'的',u'〔',u'〕',u'〝',u'〞',u'『 ',u'』',u'〈',u'〉',u'\\',u'（', u'）',u'～',u'u3000',u'"',u'&gt',u'&lt',u';',u'，', u'。', u'、', u'；', u'：', u'?', u'「', u'」', u'%', u'.', u',', u'？', u'-', u'~',u'!',u'！', u'&nbsp', u'<BR>', u'“', u'”', u'【', u'】', u'《', u'》',u'：',u'●',u'(',u')',u'；']
        for chin in chineseFilter1:
            re[times] = str(re[times])#轉為字串才能處理
            re[times] = re[times].replace(chin,'')
        re_cut = jieba.cut(re[times])
        re_cut_show = " ".join(re_cut)##空白一定要有不然她無法斷後面的token
        #print(re_cut_show)
        #re[times] = re_cut_show ##把結果丟回去
        re_token = nltk.word_tokenize(re_cut_show)
        #print(re_token)
        #re_token_freq = nltk.FreqDist(re_token)
        re[times] = re_token

def new_gene_list():
    for cani_num in range(0,len(results)):
        #print('now cadina=',results[cani_num])
        tem_gene=[]
        for i in range(0,len(results[cani_num])):
                #print('現在token到:',results[cani_num][i], '第', i , '個')
                #tem_gene=[]
                for j in range(0,len(spilt_s)):
                    #print('現在check到:',spilt_s[j], '第', j , '個')
                    if spilt_s[j] == results[cani_num][i]:
                        tem_gene.append(spilt_s[j])
                        #print('spilt_s=',spilt_s[j],', test_token=',results[cani_num][i])
                #print('tem_gene=',tem_gene)
        new_gene.append(tem_gene)
                
delpunc_n_jieba_n_tok(results)
new_gene_list()
#print('delete the punciation and jieba with customer dictionary')
#print(results)
#print(new_gene)
#print(len(new_gene))
print('-----finish-----')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Leaves\AppData\Local\Temp\jieba.cache


-----start-----
import the customer dictionary....


Loading model cost 1.027 seconds.
Prefix dict has been built succesfully.


*sccessful import*
try to link with DataBase:cate_test......
*sccessful link*
setting up the data.......
*sccessful set up*
-----finish-----


In [9]:
#一key一value的建立字典函數
def create_dictione_one_key_one_value(text_path):
    dict_data_create = {}##生成完成的字典
    dict_key = {}##暫存
    dict_key['whatever']="just_for_test"
    with open(text_path, encoding = 'utf8') as df:
        for kv in [d.strip().split('	') for d in df]:
            #print('kv[0]=',kv[0])
            if kv[0] not in dict_key['whatever']:
                dict_key['whatever'] = kv[0]      
            #print('kv[1]=',kv[1])
            #print()
            dict_data_create.setdefault(kv[0], [])
            if len(dict_data_create[kv[0]])<1: #不重複放對應值進去
                dict_data_create[kv[0]].append(kv[1])
    return dict_data_create

####詞替換
####list中的List
def word_to_specified(orig_data,speci_dict):
    count_candi = 1#算第幾個候選人
    input_to_output_list = []
    for candi_list in orig_data:
        print('第',count_candi, '個候選人的原始資料->', candi_list)
        temporary_candi= []#每個候選人的暫存
        for word_in_cada_list in candi_list:
            if word_in_cada_list in speci_dict.keys():
                print('塞->',speci_dict[word_in_cada_list][0])
                temporary_candi.append(speci_dict[word_in_cada_list][0])
        print('第',count_candi,'個候選人更改成類別成功->',temporary_candi)
        input_to_output_list.append(temporary_candi)
        count_candi = count_candi+1
        print()
    return input_to_output_list

#把全部new_gene的key算詞頻
#這一趴出來還是NLTK的freqdist
def count_nltk_frequent_of_key(k_li):
    key_freq_list=[]
    for k_li_num in k_li:
        #print(k_li_num)
        k_li_tem_freq = nltk.FreqDist(k_li_num)
        key_freq_list.append(k_li_tem_freq)
    return key_freq_list

##轉成一般
def convert_normal_freq_to_dict(k_f_list):
    ues_this_k_real_list=[]
    for k_f_list_dic_freq in range(0,len(k_f_list)):
        k_real_freq={}
        k_real_freq.update(k_f_list[k_f_list_dic_freq])
        #print(k_real_freq)
        ues_this_k_real_list.append(k_real_freq)
    return ues_this_k_real_list

#算全部詞頻(用一個一個候選人的去累計)
def count_all_fre_data_from(def_in_this_k_real_list):
    count_fre_data_from = {}
    for cadina_in_ues in def_in_this_k_real_list:
        #print('現在到哪個候選人->', cadina_in_ues)
        for cadina_key in cadina_in_ues.keys():
            #print('到哪個key->', cadina_key)
            if cadina_key not in count_fre_data_from.keys():
                #print('沒有在新詞頻字典')
                count_fre_data_from[cadina_key] = cadina_in_ues[cadina_key] 
            else:
                #print('有在')
                count_fre_data_from[cadina_key] = count_fre_data_from[cadina_key] + cadina_in_ues[cadina_key]
        #print('現在的新詞頻辭典', count_fre_data_from)
    return count_fre_data_from

##建矩陣用的列表
def create_list_for_generate_matrix(list_in_word_path):
    with open(list_in_word_path,encoding = 'utf8') as f:
        mylist = [line.rstrip('\n') for line in f]
    return mylist

##減肥
###給各因素裡的小朋友用的
def remove_repeat_in_list(repeat_list):
    all_removed_each_reapeat_list = []
    for each_reapeat_list in repeat_list:
        print('減肥前->',each_reapeat_list)
        removed_each_reapeat_list = list(set(each_reapeat_list))
        print('減肥後->',removed_each_reapeat_list)
        print()
        all_removed_each_reapeat_list.append(removed_each_reapeat_list)#把暫存塞到所有裡
    return all_removed_each_reapeat_list



In [5]:
###生字典

dict_of_origin_to_theme = create_dictione_one_key_one_value('0721try.txt')
dict_of_origin_to_cate = create_dictione_one_key_one_value('0711cate.txt')
dict_of_theme_to_cate = create_dictione_one_key_one_value('from_categ_to_theme.txt')

In [6]:
##換詞

get_orig_to_cate_list = word_to_specified(new_gene, dict_of_origin_to_cate)
get_orig_to_theme_list = word_to_specified(new_gene, dict_of_origin_to_theme)

第 1 個候選人的原始資料-> ['公托中心', '公托中心', '產學合作', '就業創業']
塞-> 幼托
塞-> 幼托
塞-> 教育
塞-> 就業
第 1 個候選人更改成類別成功-> ['幼托', '幼托', '教育', '就業']

第 2 個候選人的原始資料-> ['公民投票', '督促政府', '青年發展', '青年創業', '就業環境', '教育改革', '品格教育', '國土規劃', '資源合理分配']
塞-> 民權
塞-> 監察
塞-> 青年
塞-> 青年
塞-> 就業
塞-> 教育
塞-> 教育
塞-> 國家定位
塞-> 正義
第 2 個候選人更改成類別成功-> ['民權', '監察', '青年', '青年', '就業', '教育', '教育', '國家定位', '正義']

第 3 個候選人的原始資料-> ['財團壟斷', '營養午餐', '弱勢族群', '保障人民', '生命財產', '產業升級', '保障人民']
塞-> 產業
塞-> 民生
塞-> 弱勢
塞-> 民權
塞-> 民生
塞-> 經濟
塞-> 民權
第 3 個候選人更改成類別成功-> ['產業', '民生', '弱勢', '民權', '民生', '經濟', '民權']

第 4 個候選人的原始資料-> ['十二年國教', '監督政府', '立法保障', '監督政府', '都市更新', '立法保障', '中小企業', '立法保障']
塞-> 教育
塞-> 監察
塞-> 立法
塞-> 監察
塞-> 建設
塞-> 立法
塞-> 經濟
塞-> 立法
第 4 個候選人更改成類別成功-> ['教育', '監察', '立法', '監察', '建設', '立法', '經濟', '立法']

第 5 個候選人的原始資料-> ['世代正義', '青年創業', '勞工退休', '勞工退休', '居住正義', '社會住宅', '區段徵收', '地下停車場', '青年創業']
塞-> 正義
塞-> 青年
塞-> 就業
塞-> 就業
塞-> 民生
塞-> 建設
塞-> 建設
塞-> 建設
塞-> 青年
第 5 個候選人更改成類別成功-> ['正義', '青年', '就業', '就業', '民生', '建設', '建設', '建設', '青年']

第 6 個候選人的原始資料-> ['長期照護', '就業機會

In [12]:
##先轉nltk詞頻再轉正常詞頻(各個候選人)

get_origin_to_cate_nltk_freq = count_nltk_frequent_of_key(get_orig_to_cate_list)
get_origin_to_theme_nltk_freq = count_nltk_frequent_of_key(get_orig_to_theme_list)

get_origin_to_cate_freq = convert_normal_freq_to_dict(get_origin_to_cate_nltk_freq)
get_origin_to_theme_freq = convert_normal_freq_to_dict(get_origin_to_theme_nltk_freq)


In [13]:
###加碼算整個資料表的

get_all_table_origin_to_cate_freq = count_all_fre_data_from(get_origin_to_cate_freq)
get_all_table_origin_to_theme_freq = count_all_fre_data_from(get_origin_to_theme_freq)

{'18歲公民權': 8,
 '世代正義': 10,
 '世界大同': 11,
 '中低收入戶': 13,
 '中央經費': 2,
 '中小企業': 19,
 '中產階級': 6,
 '中華民國在台灣': 2,
 '人民作主': 27,
 '人民參與': 2,
 '人民服務': 6,
 '企業獲利': 6,
 '企業發展': 3,
 '住房問題': 5,
 '促進產業': 6,
 '保護台灣': 2,
 '保障人民': 10,
 '保障勞工': 6,
 '保障原住民': 1,
 '保障農民': 1,
 '健保免費': 23,
 '偽中華民國': 6,
 '傳統產業': 7,
 '債留子孫': 4,
 '兩岸協議': 1,
 '兩岸協議監督': 5,
 '兩岸和平': 8,
 '公共建設': 5,
 '公共托育': 15,
 '公園綠地': 7,
 '公平升學': 1,
 '公平合理': 3,
 '公平教育': 6,
 '公平正義': 47,
 '公民投票': 7,
 '公開透明': 24,
 '再生能源': 5,
 '制度推動': 2,
 '創造就業': 10,
 '加入聯合國': 8,
 '勞動基準': 6,
 '勞工就業': 4,
 '勞工權益': 22,
 '勞工退休': 4,
 '區段徵收': 8,
 '十二年國教': 8,
 '協助中小企業': 7,
 '原住民族': 4,
 '原民自治': 1,
 '參與政治': 6,
 '台灣主權': 29,
 '台灣新政治': 5,
 '台灣獨立': 57,
 '台灣經濟': 14,
 '台灣農工': 6,
 '司法改革': 17,
 '吃得安心': 7,
 '合理分配': 3,
 '品格教育': 8,
 '嚴格監督': 6,
 '國土規劃': 13,
 '國家主權': 4,
 '國會議事': 5,
 '國營企業': 12,
 '國際市場': 8,
 '國際社會': 6,
 '國際競爭力': 7,
 '國際觀光': 4,
 '在地就業': 6,
 '在地產業': 5,
 '在地經濟': 12,
 '在地老化': 5,
 '地下停車': 1,
 '地下停車場': 8,
 '地方發展': 4,
 '基本工資': 18,
 '基礎建設': 4,
 '增加就業': 2,
 '多元文化': 7,
 '大眾運輸': 7,
 '學